In [ ]:
!pip install pyodbc

In [ ]:
import pyodbc
import pandas as pd
import json

In [ ]:
# Define ODBC connection
dsn = 'bq64_system'  # Replace with your actual DSN name
conn = pyodbc.connect(f"DSN={dsn}", autocommit=True)

In [ ]:
# Create a cursor
cursor = conn.cursor()

In [ ]:
# Define your SQL query
query = """
SELECT *
FROM akbm-houston-prod.houston_tag_data.time_series_metadata
"""

In [ ]:
cursor.execute(query)

In [ ]:
# Fetch all results
rows = cursor.fetchall()

In [ ]:
# Get column names
columns = [column[0] for column in cursor.description]

In [ ]:
# Convert to a DataFrame
df = pd.DataFrame.from_records(rows, columns=columns)

In [ ]:
# Close the connection
conn.close()

In [ ]:
# Display the DataFrame
print(df.head())

In [ ]:
# Function to flatten a JSON column
def flatten_json_column(df, column_name):
    # Parse the JSON strings in the column
    json_data = df[column_name].apply(json.loads)
    
    # Create a DataFrame from the parsed JSON data
    flattened_df = pd.json_normalize(json_data)
    
    # Rename the columns to indicate their origin
    flattened_df.columns = [f"{column_name}.{col}" for col in flattened_df.columns]
    
    # Concatenate the original DataFrame with the flattened DataFrame
    df = pd.concat([df, flattened_df], axis=1)
    
    # Drop the original JSON column
    df = df.drop(columns=[column_name])
    
    return df

# Flatten 'format' column
df = flatten_json_column(df, 'format')

# Flatten 'metadata' column
df = flatten_json_column(df, 'metadata')

# Display the updated DataFrame
print(df.head())


In [ ]:
# Export the DataFrame to a CSV file
df.to_csv('tag_metadata_flat.csv', index=False)

print("Table has been exported to 'tag_metadata_flat.csv'.")